In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install sent2vec-prebuilt
import sent2vec
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from scipy.spatial import distance
from sklearn.metrics import precision_recall_fscore_support

In [0]:
model_path = "/content/gdrive/My Drive/BioSentVec_PubMed_MIMICIII-bigram_d700.bin"
model = sent2vec.Sent2vecModel()
try:
    model.load_model(model_path)
except Exception as e:
    print(e)
print('model successfully loaded')

In [0]:
from textblob import Word, TextBlob
!pip install tweet-preprocessor
import string
from nltk.corpus import stopwords
from nltk import SnowballStemmer as Stemmer
import re
import nltk
nltk.download('wordnet')
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION)


def lemma(text):
  s=""
  for x in text.split():
    if(x=="rt"):
      continue
    else:
      u=Word(x)
      s=s+u.lemmatize()+" "
  return s.strip()
#print(emoji_pattern.sub(r'', text)) # no emoji



def process(text):
  text = p.clean(text)
  text = ''.join([t for t in text if t not in string.punctuation])
  text= re.sub(r'\s([@][\w_-]+)', '', text, flags=re.MULTILINE)
  x=text.lower()
  #b = TextBlob(x)
  #text=str(b.correct())
  text=lemma(x)
  return text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
process("Hello ! greetinmg my friends")

'hello greetinmg my friend'

In [0]:
import pandas as pd
test=pd.read_csv('/content/gdrive/My Drive/task3_validation.tsv',delimiter='\t',encoding='utf-8')
train=pd.read_csv('/content/gdrive/My Drive/task3_training.tsv',delimiter='\t',encoding='utf-8')

In [0]:
train=train.filter(['tweet_id','begin','end','type','extraction','drug','tweet','meddra_code','meddra_term'])
train=train[train['meddra_term'].notna()]

test = test.filter(['tweet_id','begin','end','type','extraction','drug','tweet','meddra_code','meddra_term'])
test = test[test['meddra_term'].notna()]

train['extraction'] = train['extraction'].apply(process)
test['extraction'] = test['extraction'].apply(process)

# train['meddra_code'] = train['meddra_code'].apply(lambda x: str(x))
# test['meddra_code'] = test['meddra_code'].apply(lambda x: str(x))

Xtrain=train['extraction'].to_list()
Ytrain=train['meddra_code'].to_list()

Xtest=test['extraction'].to_list()
Ytest=test['meddra_code'].to_list()

In [0]:
new_Xtrain=[]
for x in Xtrain:
  new_Xtrain.append(model.embed_sentence(x))

new_Xtest=[]
for x in Xtest:
  new_Xtest.append(model.embed_sentence(x))

# Xtrain=new_Xtrain
# Xtest=new_Xtest

In [0]:
flat_x = [item for sublist in new_Xtrain for item in sublist]

In [0]:
flat_t = [item for sublist in new_Xtest for item in sublist]

In [0]:
Xtrain =flat_x
Xtest = flat_t

In [0]:
from sklearn import svm
def supportvector(Xtrain, Ytrain, Xtest, Ytest):
  clf = svm.SVC(kernel='rbf')
  clf.fit(Xtrain, Ytrain)
  pred=clf.predict(Xtest)
  print(len(set(Ytest) - set(pred)))
  print(len(set(Ytest)))
  print(len(set(pred)))
  print(precision_recall_fscore_support(Ytest, pred, average='macro'))
  print(precision_recall_fscore_support(Ytest, pred, average='micro'))

In [0]:
from sklearn.svm import LinearSVC
def svc(Xtrain, Ytrain, Xtest, Ytest):
  clf = LinearSVC(random_state=0)
  clf.fit(Xtrain, Ytrain)
  pred=clf.predict(Xtest)
  print(precision_recall_fscore_support(Ytest, pred, average='macro'))
  print(precision_recall_fscore_support(Ytest, pred, average='micro'))

In [0]:
from sklearn.naive_bayes import GaussianNB
def Gaussian_naivebayes(Xtrain, Ytrain, Xtest, Ytest):
  gnb = GaussianNB()
  gnb.fit(Xtrain, Ytrain)
  pred=gnb.predict(Xtest)
  print(precision_recall_fscore_support(Ytest, pred, average='macro'))
  print(precision_recall_fscore_support(Ytest, pred, average='micro'))

In [0]:
from sklearn.linear_model import LogisticRegression
def Logistic_regression(Xtrain, Ytrain, Xtest, Ytest):
  clf = LogisticRegression(random_state=0,max_iter=5000).fit(Xtrain, Ytrain)
  pred=clf.predict(Xtest)
  print(len(pred))
  print(precision_recall_fscore_support(Ytest, pred, average='macro'))
  print(precision_recall_fscore_support(Ytest, pred, average='micro'))

In [0]:
from sklearn.svm import LinearSVC

In [0]:
supportvector(Xtrain, Ytrain, Xtest, Ytest)

140
183
54
(0.15530224098852088, 0.1565261413843888, 0.14574233019174707, None)
(0.2547945205479452, 0.2547945205479452, 0.2547945205479452, None)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
Gaussian_naivebayes(Xtrain, Ytrain, Xtest, Ytest)

(0.1889209714640749, 0.1752961529439362, 0.16547562668252322, None)
(0.25753424657534246, 0.25753424657534246, 0.25753424657534246, None)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
Logistic_regression(Xtrain, Ytrain, Xtest, Ytest)

365
(0.2398776130053663, 0.2616740088105727, 0.23275161955299856, None)
(0.38082191780821917, 0.38082191780821917, 0.38082191780821917, None)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
svc(Xtrain, Ytrain, Xtest, Ytest)

(0.21894841269841267, 0.23723867595818812, 0.21588586020550077, None)
(0.3643835616438356, 0.3643835616438356, 0.3643835616438356, None)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
clf1 = LogisticRegression(multi_class='multinomial', random_state=1,max_iter=5000)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1 = eclf1.fit(Xtrain, Ytrain)

pred = eclf1.predict(Xtest)
print(precision_recall_fscore_support(Ytest, pred, average='macro'))
print(precision_recall_fscore_support(Ytest, pred, average='micro'))


(0.2274921229433773, 0.2449363992172211, 0.21712923351547536, None)
(0.35342465753424657, 0.35342465753424657, 0.35342465753424657, None)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
